In [ ]:
#we install the transformers library and pytorch
#this is the general workflow for modern deep learning type projects

!pip install transformers[torch]

In [32]:
from transformers import AutoTokenizer, AutoModel

#distilbert is a smaller version of bert, but trained on the same data and optimized
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

In [26]:
#we can now tokenize a sequence of text

tokenized_sequence = tokenizer.tokenize("test blargeldy their, they're")


In [27]:
#notice how bert handles contractions and unseen words
tokenized_sequence

['test', 'b', '##lar', '##gel', '##dy', 'their', ',', 'they', "'", 're']

In [36]:
input

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [38]:
input=tokenizer("Hello, this one sentence!", "And this sentence goes with it.", return_tensors='pt')

output=model(**input)

In [70]:
#value for the classification token, we can use this to do downstream nlp tasks
print(output.last_hidden_state[0][0])
print(output.last_hidden_state[0][0].shape)

tensor([-1.3919e-01, -7.1982e-02,  4.0344e-02, -2.2243e-01, -1.7465e-01,
        -1.6524e-01,  2.8922e-01,  3.8080e-01, -2.0477e-01, -1.9203e-01,
        -8.0516e-02, -6.3089e-02, -2.2332e-01,  1.5912e-01,  1.7414e-01,
         1.2588e-01, -8.3952e-02,  1.9253e-01,  1.1660e-01, -1.1700e-01,
        -1.6660e-01, -8.2792e-02,  3.4711e-02, -4.5443e-02,  3.5760e-02,
        -6.7369e-02, -1.0968e-01, -6.6397e-02, -4.2676e-02,  1.0038e-02,
        -7.6260e-03,  1.0621e-01, -8.4307e-02, -5.1339e-02,  3.1836e-02,
         1.4903e-02,  7.4059e-02, -8.4516e-03,  1.0604e-01, -1.2139e-02,
        -1.7827e-01,  8.6443e-02,  1.7273e-01, -2.7619e-02, -8.3613e-02,
        -2.3327e-01, -2.4096e+00, -7.7631e-02, -2.2396e-01, -2.3699e-01,
         9.3535e-02,  5.2516e-02, -4.8690e-02,  2.9909e-01,  1.2537e-01,
         2.2622e-01, -4.0399e-01,  3.3692e-01, -4.6337e-02, -3.3084e-02,
         1.3035e-01, -3.1199e-02, -9.2113e-03,  9.7409e-02, -9.4188e-02,
         1.8251e-01,  2.2600e-02,  2.8169e-01, -3.9

In [78]:
input=tokenizer("She withdrew money from the bank.", "She fished by the river bank.", return_tensors='pt')

output=model(**input)

In [79]:
#calculating the cosine similarity between the two sentences
import torch.nn.functional as F

cosine_similarity = F.cosine_similarity(output.last_hidden_state[0][6], output.last_hidden_state[0][14], dim=0)
print(cosine_similarity)

tensor(0.8869, grad_fn=<SumBackward1>)


In [80]:
input=tokenizer("She withdrew money from the bank.", "She made a deposit at the bank.", return_tensors='pt')
output=model(**input)
cosine_similarity = F.cosine_similarity(output.last_hidden_state[0][6], output.last_hidden_state[0][15], dim=0)
print(cosine_similarity)

tensor(0.9662, grad_fn=<SumBackward1>)


In [43]:
#there are tons of premade and pretrained pipelines in huggingface
from transformers import pipeline
unmasker = pipeline('fill-mask', model="distilbert-base-uncased")

unmasker("Hello I'm a [MASK] model.")

[{'score': 0.05292845517396927,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello i ' m a role model."},
 {'score': 0.03968577831983566,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello i ' m a fashion model."},
 {'score': 0.034743569791316986,
  'token': 2449,
  'token_str': 'business',
  'sequence': "hello i ' m a business model."},
 {'score': 0.03462294116616249,
  'token': 2944,
  'token_str': 'model',
  'sequence': "hello i ' m a model model."},
 {'score': 0.01814522221684456,
  'token': 11643,
  'token_str': 'modeling',
  'sequence': "hello i ' m a modeling model."}]

In [44]:
# we can also examine some biases in the model
unmasker("The man worked as a [MASK].")

[{'score': 0.11590294539928436,
  'token': 10533,
  'token_str': 'carpenter',
  'sequence': 'the man worked as a carpenter.'},
 {'score': 0.0723821222782135,
  'token': 20987,
  'token_str': 'blacksmith',
  'sequence': 'the man worked as a blacksmith.'},
 {'score': 0.05382770299911499,
  'token': 22701,
  'token_str': 'tailor',
  'sequence': 'the man worked as a tailor.'},
 {'score': 0.0413505956530571,
  'token': 15610,
  'token_str': 'waiter',
  'sequence': 'the man worked as a waiter.'},
 {'score': 0.03554900735616684,
  'token': 14998,
  'token_str': 'butcher',
  'sequence': 'the man worked as a butcher.'}]

In [45]:
unmasker("The woman worked as a [MASK].")

[{'score': 0.12517952919006348,
  'token': 6821,
  'token_str': 'nurse',
  'sequence': 'the woman worked as a nurse.'},
 {'score': 0.08857159316539764,
  'token': 10850,
  'token_str': 'maid',
  'sequence': 'the woman worked as a maid.'},
 {'score': 0.07708469033241272,
  'token': 13877,
  'token_str': 'waitress',
  'sequence': 'the woman worked as a waitress.'},
 {'score': 0.054324790835380554,
  'token': 10533,
  'token_str': 'carpenter',
  'sequence': 'the woman worked as a carpenter.'},
 {'score': 0.04624558985233307,
  'token': 22583,
  'token_str': 'housekeeper',
  'sequence': 'the woman worked as a housekeeper.'}]

In [63]:
unmasker("A person from [MASK] is an enemy.")

[{'score': 0.027150411158800125,
  'token': 6014,
  'token_str': 'heaven',
  'sequence': 'a person from heaven is an enemy.'},
 {'score': 0.016995996236801147,
  'token': 28978,
  'token_str': 'afar',
  'sequence': 'a person from afar is an enemy.'},
 {'score': 0.012630046345293522,
  'token': 3607,
  'token_str': 'russia',
  'sequence': 'a person from russia is an enemy.'},
 {'score': 0.011639977805316448,
  'token': 3577,
  'token_str': 'spain',
  'sequence': 'a person from spain is an enemy.'},
 {'score': 0.010551409795880318,
  'token': 7041,
  'token_str': 'afghanistan',
  'sequence': 'a person from afghanistan is an enemy.'}]

In [65]:
unmasker("A person from [MASK] is lazy.")

[{'score': 0.019114704802632332,
  'token': 6014,
  'token_str': 'heaven',
  'sequence': 'a person from heaven is lazy.'},
 {'score': 0.01292795967310667,
  'token': 3088,
  'token_str': 'africa',
  'sequence': 'a person from africa is lazy.'},
 {'score': 0.011858571320772171,
  'token': 28978,
  'token_str': 'afar',
  'sequence': 'a person from afar is lazy.'},
 {'score': 0.009887747466564178,
  'token': 7880,
  'token_str': 'nowhere',
  'sequence': 'a person from nowhere is lazy.'},
 {'score': 0.009876523166894913,
  'token': 2634,
  'token_str': 'india',
  'sequence': 'a person from india is lazy.'}]

In [68]:
sentiment_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [69]:
#notice that our model doesn't need any preprocessing anymore
sentiment_model("I love this movie!")

[{'label': 'POS', 'score': 0.9926134347915649}]

In [83]:
import pandas as pd

In [87]:
poems=pd.read_csv("kaggle_poem_dataset.csv").head(100)

In [93]:

sentiment_model(poems["Content"][0])

[{'label': 'POS', 'score': 0.9428346157073975}]

In [97]:
#short context window, so only keep first part of each poem

poems["Content"]=poems["Content"].apply(lambda x:x[:300])

sentiments=[]
for poem in poems["Content"]:
  sentiments.append(sentiment_model(poem))

In [101]:
#extract labels from sentiments

sentiments=[sentiment[0]["label"] for sentiment in sentiments]
poems["Sentiment"]=sentiments

In [102]:
poems

,Unnamed: 0,Author,Title,Poetry Foundation ID,Content,Sentiment
0,0,Wendy Videlock,!,55489,"Dear Writers, I’m compiling the first in what ...",POS
1,1,Hailey Leithauser,0,41729,"Philosophic\nin its complex, ovoid emptiness,\...",NEU
2,2,Jody Gladding,1-800-FEAR,57135,We'd like to talk with you about fear t...,NEU
3,3,Joseph Brodsky,1 January 1965,56736,The Wise Men will unlearn your name.\nAbove yo...,NEU
4,4,Ted Berrigan,3 Pages,51624,For Jack Collom\n10 Things I do Every Day\n\np...,NEU
...,...,...,...,...,...,...
95,95,Myung Mi Kim,[accumulation of land]\n \n \n \n Launch Au...,53678,accumulation of land maintain hou...,NEU
96,96,Cally Conan-Davies,Ace,55551,"Bloody hell, the world’s turned\nupside down\n...",NEG
97,97,Robert Gernhardt,Ach/Last Call,40352,Right up to my final hour\nI'll be obliging an...,NEU
98,98,Denise Levertov,The Ache of Marriage,42523,"The ache of marriage:\n\nthigh and tongue, bel...",NEU


In [151]:
!pip install -U sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [152]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']

# Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)

c:\Users\lucia\Documents\GitHub\huggingface-demo\.conda\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lucia\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [156]:
#calculate cosine similarity between two sentences

from sklearn.metrics.pairwise import cosine_similarity

sentence1 = "I love apples"
sentence2 = "I hate apples"
# Encode sentences to get their embeddings

embedding1 = model.encode(sentence1)

embedding2 = model.encode(sentence2)

# Compute cosine similarity between the two sentences

cosine_similarity([embedding1], [embedding2])

array([[0.6667714]], dtype=float32)

In [104]:
#huggingface also has some built in datasets
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   --- ------------------------------------ 2.4/25.1 MB 12.2 MB/s eta 0:00:02
   ------- -------------------------------- 4.5/25.1 MB 11.2 MB/s eta 0:00:02
   ---------- ----------------------------- 6.8/25.1 MB 11.7 MB/s eta 0:00:02
   -------------- ------------------------- 8.9/25.1 MB 11.1 MB/s eta 0:00:02
   ----------------- ---------------------- 11.3/25.1 MB 11.2 MB/s eta 0:00:02
   ---------------------- ----------------- 13.9/25.1 MB 11.2 MB/s eta 0:00:02
   ------------------------- -------------- 16.0/25.1 MB 11.2 MB/s eta 0:00:01
   ----------------------------- ---------- 18.4/25.1 MB 11.2 MB/s eta 0:00:01
   -------------------------------- ------- 20.7/25.1 MB 11.2 MB/s eta 0:00:01
   ----------------------------------- ---- 22.5/25.1 MB 11.1 MB/s eta 0:0

In [108]:
#makes sure all input are the same length
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [121]:
#load the dataset

uiuc=pd.read_csv("uiuc.csv")
mich=pd.read_csv("umich.csv")

#sample so we have even number of samples from each dataset
mich=mich.sample(n=4725)

#assign labels based on origin subreddit of comment
uiuc['label']=1
mich['label']=0

#you will be working with the data csv for the rest of the question
data=pd.concat([uiuc,mich])


In [116]:
from datasets import Dataset
dataset=Dataset.from_pandas(data[['text','label']])

In [118]:
dataset=dataset.train_test_split(test_size=0.2)

In [119]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 1890/1890 [00:00<00:00, 8210.29 examples/s]


In [120]:
tokenized_datasets["train"][0]

{'text': 'It’s a good tip for your entire lives.  There is no drawback. \n\nThat said, the profound discovery that daily bathing, much less wearing deodorant, is entirely avoided by a concerning portion of students who pile into the buses, will give a young person a sobering introduction to life’s weirdness if they hadn’t already experienced it before.',
 'label': 1,
 '__index_level_0__': 431,
 'input_ids': [101,
  2009,
  1521,
  1055,
  1037,
  2204,
  5955,
  2005,
  2115,
  2972,
  3268,
  1012,
  2045,
  2003,
  2053,
  4009,
  5963,
  1012,
  2008,
  2056,
  1010,
  1996,
  13769,
  5456,
  2008,
  3679,
  17573,
  1010,
  2172,
  2625,
  4147,
  2139,
  7716,
  18842,
  2102,
  1010,
  2003,
  4498,
  9511,
  2011,
  1037,
  7175,
  4664,
  1997,
  2493,
  2040,
  8632,
  2046,
  1996,
  7793,
  1010,
  2097,
  2507,
  1037,
  2402,
  2711,
  1037,
  17358,
  2075,
  4955,
  2000,
  2166,
  1521,
  1055,
  6881,
  2791,
  2065,
  2027,
  2910,
  1521,
  1056,
  2525,
  5281,
  2

In [122]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [123]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
!pip install evaluate

In [129]:
import numpy as np
import evaluate

#load accuracy evaluation
metric = evaluate.load("accuracy")

In [130]:
from transformers import TrainingArguments, Trainer

#default training parameters, don't worry about these for now
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [133]:
#calculate the accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [144]:
#combine all previous defined parameters into a trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [145]:
#run training
trainer.train()

  0%|          | 0/3 [01:04<?, ?it/s]


KeyboardInterrupt: 